In [ ]:
!pip install openai pandas
!pip install python-dotenv


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: C:\Users\user\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: C:\Users\user\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip


In [6]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
print("API 키 앞 10자리:", api_key[:10])

API 키 앞 10자리: sk-proj--a


In [ ]:
# GPT-4o 모델을 사용하여 보이스피싱 대화 시나리오(대출빙자형, 메신저 피싱형, 택배사칭형)를 자동으로 생성하고, 
# 생성된 데이터를 중복 없이 CSV 파일로 저장하는 자동화 파이프라인입니다.
# 각 대화는 3~10문장으로 구성되며, 피싱범(0)과 피해자(1)가 번갈아가며 대화하고, 한 대화에는 한 가지 유형만 포함됩니다.
# 대화 파일명은 phishing_001처럼 순차적으로 부여되고, 모든 대화에서 이름은 'OOO'만 사용하며, 실제적이고 자연스러운 시나리오만 생성됩니다.
# 생성된 데이터는 기존 CSV와 합쳐 중복을 제거한 뒤 저장됩니다.
# RateLimitError 등 예외 상황도 처리합니다.

import os
import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI, RateLimitError

# 1. 환경 변수 로드 및 API 키 확인
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("❌ API 키가 로드되지 않았습니다. .env 파일을 확인하세요.")

client = OpenAI(api_key=api_key)

# 2. 시스템 프롬프트 (영어, 출력은 반드시 한국어)
system_prompt = """
You are a financial crime investigation expert with 15 years of experience and a developer of voice phishing detection systems.
Please create realistic and natural phone call scenarios for building a voice phishing detection dataset.

Rules:
1. Each conversation consists of 3 to 10 sentences, with a balanced distribution of sentence counts across conversations.
2. The phishing scammer (0) and the victim (1) alternate in the conversation.
3. Only generate three types of conversations: loan fraud, messenger phishing, and delivery impersonation.
4. Each conversation should contain only one type.
5. Use the name 'OOO' exclusively in all conversations (e.g., "Hello, is this OOO? This is regarding a loan.").
6. Make the conversations realistic and natural without duplication. Avoid overlapping with previously generated conversations.
7. Output the results in the following CSV format (without headers):

phishing_001,loan fraud,0,Hello, is this OOO? I am contacting you regarding a loan.
phishing_001,loan fraud,1,Yes? What is this about?
(Generate 5 conversations from phishing_001 to phishing_005.)

Please output the conversation text in Korean.
"""

# 3. GPT-4o로 시나리오 생성
def generate_phishing_dialogues(n, start_index, conversation_type):
    # conversation_type을 영어로 변환
    type_en = {
        "대출빙자형": "loan fraud",
        "메신저 피싱형": "messenger phishing",
        "택배사칭형": "delivery impersonation"
    }[conversation_type]
    user_prompt = (
        f"Please generate {n} {type_en} voice phishing scenarios. "
        f"File names should be assigned sequentially from phishing_{start_index:03d}."
    )
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.8
    )
    return response.choices[0].message.content

# 4. 생성 결과를 CSV에 누적 저장(중복 제거)
def parse_and_save_csv_append_unique(response_text, save_path="./dataset/phishing_data.csv"):
    lines = [line.strip() for line in response_text.strip().split("\n") if line and not line.startswith("file_name")]
    data = [line.split(",", maxsplit=3) for line in lines if len(line.split(",", maxsplit=3)) == 4]
    df_new = pd.DataFrame(data, columns=["file_name", "conversation_type", "spk", "msg"])
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    if os.path.exists(save_path):
        df_existing = pd.read_csv(save_path)
        df_combined = pd.concat([df_existing, df_new], ignore_index=True)
        df_combined.drop_duplicates(inplace=True)
        df_combined.to_csv(save_path, index=False, encoding="utf-8-sig")
    else:
        df_new.to_csv(save_path, index=False, encoding="utf-8-sig")
    print(f"✅ CSV 저장 완료 및 중복 제거: {save_path}")

# 5. 실행
if __name__ == "__main__":
    print("🔄 보이스피싱 대화 생성 중...")
    try:
        total_count = 300
        batch_size = 5
        conversation_types = ["대출빙자형", "메신저 피싱형", "택배사칭형"]
        count_per_type = total_count // len(conversation_types)  # 100개씩
        start_index = 1
        for ctype in conversation_types:
            for i in range(0, count_per_type, batch_size):
                print(f"생성 중: {ctype} {i+1} ~ {i+batch_size} (파일명 시작: phishing_{start_index:03d})")
                output = generate_phishing_dialogues(batch_size, start_index, ctype)
                parse_and_save_csv_append_unique(output, save_path="./dataset/phishing_data.csv")
                start_index += batch_size
    except RateLimitError:
        print("❌ RateLimitError: 쿼터 초과 또는 모델 접근 제한입니다. 잠시 후 재시도하거나 대시보드를 확인하세요.")
    except Exception as e:
        print(f"❌ 오류 발생: {e}")


🔄 보이스피싱 대화 생성 중...
생성 중: 대출빙자형 1 ~ 5 (파일명 시작: phishing_001)
✅ CSV 저장 완료 및 중복 제거: ./dataset/phishing_data.csv
생성 중: 대출빙자형 6 ~ 10 (파일명 시작: phishing_006)
✅ CSV 저장 완료 및 중복 제거: ./dataset/phishing_data.csv
생성 중: 대출빙자형 11 ~ 15 (파일명 시작: phishing_011)
✅ CSV 저장 완료 및 중복 제거: ./dataset/phishing_data.csv
생성 중: 대출빙자형 16 ~ 20 (파일명 시작: phishing_016)
✅ CSV 저장 완료 및 중복 제거: ./dataset/phishing_data.csv
생성 중: 대출빙자형 21 ~ 25 (파일명 시작: phishing_021)
✅ CSV 저장 완료 및 중복 제거: ./dataset/phishing_data.csv
생성 중: 대출빙자형 26 ~ 30 (파일명 시작: phishing_026)
✅ CSV 저장 완료 및 중복 제거: ./dataset/phishing_data.csv
생성 중: 대출빙자형 31 ~ 35 (파일명 시작: phishing_031)
✅ CSV 저장 완료 및 중복 제거: ./dataset/phishing_data.csv
생성 중: 대출빙자형 36 ~ 40 (파일명 시작: phishing_036)
✅ CSV 저장 완료 및 중복 제거: ./dataset/phishing_data.csv
생성 중: 대출빙자형 41 ~ 45 (파일명 시작: phishing_041)
✅ CSV 저장 완료 및 중복 제거: ./dataset/phishing_data.csv
생성 중: 대출빙자형 46 ~ 50 (파일명 시작: phishing_046)
✅ CSV 저장 완료 및 중복 제거: ./dataset/phishing_data.csv
생성 중: 대출빙자형 51 ~ 55 (파일명 시작: phishing_051)
✅ CSV 저장 완료 및 중복 제거: 

In [ ]:
# GPT-4o 모델을 사용하여 기관사칭형(경찰, 검찰, 은행, 세무서, 보험 등) 보이스피싱 대화 시나리오를 자동으로 생성하고, 
# 생성된 데이터를 중복 없이 CSV 파일로 저장하는 자동화 파이프라인입니다.  
# 각 대화는 3~10문장으로 구성되며, 피싱범(0)과 피해자(1)가 번갈아가며 대화하고, 한 대화에는 기관사칭형 유형만 포함됩니다.  
# 대화 파일명은 phishing_001처럼 순차적으로 부여되고, 모든 대화에서 이름은 'OOO'만 사용하며, 실제적이고 자연스러운 시나리오만 생성됩니다.  
# 기관사칭 범죄의 다양성을 위해 사칭 기관(경찰, 검찰, 은행, 세무서, 보험 등)과 피해자 반응(의심, 순응, 당황, 저항 등)이 다양하게 포함됩니다.  
# 생성된 데이터는 기존 CSV와 합쳐 중복을 제거한 뒤 저장되며, RateLimitError 등 예외 상황도 처리합니다.

# 1. 환경 변수 로드 및 API 키 확인
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("❌ API 키가 로드되지 않았습니다. .env 파일을 확인하세요.")

client = OpenAI(api_key=api_key)

# 2. 시스템 프롬프트 (기관사칭형, 영어 프롬프트, 출력은 반드시 한국어)
system_prompt = """
You are a financial crime investigation expert with 15 years of experience and a developer of voice phishing detection systems.
Please create realistic and natural phone call scenarios for building a voice phishing detection dataset.

Rules:
1. Each conversation must be of the 'authority impersonation' (기관사칭형) type only.
2. Each conversation consists of 3 to 10 sentences, and the number of sentences should be evenly distributed across the dataset.
3. The scammer (0) and the victim (1) alternate in the conversation.
4. Only one conversation type per dialogue.
5. Always use the name 'OOO' in all conversations (e.g., "Hello, is this OOO?").
6. Make the conversations realistic, natural, and highly diverse. Vary the impersonated institutions (e.g., police, prosecutors, banks, tax office, insurance, etc.) and ensure the victim's responses are also diverse (suspicious, compliant, confused, resistant, etc.).
7. Avoid duplication with previously generated conversations.
8. Output the results in the following CSV format (without headers):

phishing_001,authority impersonation,0,Hello, is this OOO? This is Inspector Kim from the Seoul Metropolitan Police.
phishing_001,authority impersonation,1,Yes, that's me. What's going on?
(Generate 5 conversations from phishing_001 to phishing_005.)

Please output the conversation text in Korean.
"""

# 3. 기존 CSV에서 마지막 file_name의 번호를 읽어오는 함수
def get_next_start_index(save_path="./dataset/phishing_data.csv"):
    if os.path.exists(save_path):
        df = pd.read_csv(save_path)
        # file_name에서 숫자 부분만 추출해서 가장 큰 번호를 찾음
        max_idx = (
            df["file_name"]
            .str.extract(r'phishing_(\d+)')[0]
            .dropna()
            .astype(int)
            .max()
        )
        return int(max_idx) + 1
    else:
        return 1

# 4. GPT-4o로 시나리오 생성
def generate_phishing_dialogues(n, start_index):
    user_prompt = (
        f"Please generate {n} authority impersonation voice phishing scenarios. "
        f"File names should be assigned sequentially from phishing_{start_index:03d}."
    )
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.8
    )
    return response.choices[0].message.content

# 5. 생성 결과를 CSV에 누적 저장(중복 제거)
def parse_and_save_csv_append_unique(response_text, save_path="./dataset/phishing_data.csv"):
    lines = [line.strip() for line in response_text.strip().split("\n") if line and not line.startswith("file_name")]
    data = [line.split(",", maxsplit=3) for line in lines if len(line.split(",", maxsplit=3)) == 4]
    df_new = pd.DataFrame(data, columns=["file_name", "conversation_type", "spk", "msg"])
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    if os.path.exists(save_path):
        df_existing = pd.read_csv(save_path)
        df_combined = pd.concat([df_existing, df_new], ignore_index=True)
        df_combined.drop_duplicates(inplace=True)
        df_combined.to_csv(save_path, index=False, encoding="utf-8-sig")
    else:
        df_new.to_csv(save_path, index=False, encoding="utf-8-sig")
    print(f"✅ CSV 저장 완료 및 중복 제거: {save_path}")

# 6. 실행
if __name__ == "__main__":
    print("🔄 기관사칭형 보이스피싱 대화 생성 중...")
    try:
        total_count = 300      # 새로 생성할 총 대화 개수
        batch_size = 5         # 한 번에 생성할 개수
        save_path = "./dataset/phishing_data.csv"
        start_index = get_next_start_index(save_path)  # 기존 데이터가 있으면 다음 번호부터 시작
        for i in range(0, total_count, batch_size):
            print(f"생성 중: {start_index} ~ {start_index+batch_size-1} (파일명 시작: phishing_{start_index:03d})")
            output = generate_phishing_dialogues(batch_size, start_index)
            parse_and_save_csv_append_unique(output, save_path=save_path)
            start_index += batch_size
    except RateLimitError:
        print("❌ RateLimitError: 쿼터 초과 또는 모델 접근 제한입니다. 잠시 후 재시도하거나 대시보드를 확인하세요.")
    except Exception as e:
        print(f"❌ 오류 발생: {e}")


🔄 기관사칭형 보이스피싱 대화 생성 중...
생성 중: 301 ~ 305 (파일명 시작: phishing_301)
✅ CSV 저장 완료 및 중복 제거: ./dataset/phishing_data.csv
생성 중: 306 ~ 310 (파일명 시작: phishing_306)
✅ CSV 저장 완료 및 중복 제거: ./dataset/phishing_data.csv
생성 중: 311 ~ 315 (파일명 시작: phishing_311)
✅ CSV 저장 완료 및 중복 제거: ./dataset/phishing_data.csv
생성 중: 316 ~ 320 (파일명 시작: phishing_316)
✅ CSV 저장 완료 및 중복 제거: ./dataset/phishing_data.csv
생성 중: 321 ~ 325 (파일명 시작: phishing_321)
✅ CSV 저장 완료 및 중복 제거: ./dataset/phishing_data.csv
생성 중: 326 ~ 330 (파일명 시작: phishing_326)
✅ CSV 저장 완료 및 중복 제거: ./dataset/phishing_data.csv
생성 중: 331 ~ 335 (파일명 시작: phishing_331)
✅ CSV 저장 완료 및 중복 제거: ./dataset/phishing_data.csv
생성 중: 336 ~ 340 (파일명 시작: phishing_336)
✅ CSV 저장 완료 및 중복 제거: ./dataset/phishing_data.csv
생성 중: 341 ~ 345 (파일명 시작: phishing_341)
✅ CSV 저장 완료 및 중복 제거: ./dataset/phishing_data.csv
생성 중: 346 ~ 350 (파일명 시작: phishing_346)
✅ CSV 저장 완료 및 중복 제거: ./dataset/phishing_data.csv
생성 중: 351 ~ 355 (파일명 시작: phishing_351)
✅ CSV 저장 완료 및 중복 제거: ./dataset/phishing_data.csv
생성 중: 3

In [ ]:
# GPT-4o 모델을 활용하여 보이스피싱 시나리오(기관사칭형, 대출빙자형, 메신저피싱형, 택배사기형)를 자동으로 생성하고, 
# 생성된 데이터를 중복 없이 CSV 파일로 저장하는 자동화 파이프라인입니다.  
# 각 시나리오 유형별로 문장 수(기관사칭/대출빙자/메신저피싱: 8~12문장, 택배사기: 13~15문장)를 다르게 설정하며, 
# 피싱범(0)과 피해자(1)가 번갈아 대화하고, 한 대화에는 한 가지 유형만 포함됩니다.  
# 대화 파일명은 phishing_001처럼 순차적으로 부여되고, 모든 대화에서 이름은 'OOO'만 사용합니다.  
# 각 유형별로 사칭 기관·대출 종류·메신저·택배사 등과 피해자 반응(의심, 순응, 당황, 저항 등)이 다양하게 포함되도록 
# 프롬프트가 설계되어 있습니다.  
# 생성된 데이터는 기존 CSV와 합쳐 중복을 제거한 뒤 저장되며, RateLimitError 등 예외 상황도 처리합니다.

# 1. 환경 변수 로드 및 API 키 확인
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("❌ API 키가 로드되지 않았습니다. .env 파일을 확인하세요.")

client = OpenAI(api_key=api_key)

# 2. 시스템 프롬프트 (문장 수 8~12로, 유형별 안내)
system_prompt_dict = {
    "authority impersonation": """
You are a financial crime investigation expert with 15 years of experience and a developer of voice phishing detection systems.
Please create realistic and natural phone call scenarios for building a voice phishing detection dataset.

Rules:
1. Each conversation must be of the 'authority impersonation' (기관사칭형) type only.
2. Each conversation consists of 8 to 12 sentences, and the number of sentences should be evenly distributed across the dataset.
3. The scammer (0) and the victim (1) alternate in the conversation.
4. Only one conversation type per dialogue.
5. Always use the name 'OOO' in all conversations (e.g., "Hello, is this OOO?").
6. Make the conversations realistic, natural, and highly diverse. Vary the impersonated institutions (e.g., police, prosecutors, banks, tax office, insurance, etc.) and ensure the victim's responses are also diverse (suspicious, compliant, confused, resistant, etc.).
7. Avoid duplication with previously generated conversations.
8. Output the results in the following CSV format (without headers):

phishing_001,authority impersonation,0,Hello, is this OOO? This is Inspector Kim from the Seoul Metropolitan Police.
phishing_001,authority impersonation,1,Yes, that's me. What's going on?
(Generate 5 conversations from phishing_001 to phishing_005.)

Please output the conversation text in Korean.
""",
    "loan fraud": """
You are a financial crime investigation expert with 15 years of experience and a developer of voice phishing detection systems.
Please create realistic and natural phone call scenarios for building a voice phishing detection dataset.

Rules:
1. Each conversation must be of the 'loan fraud' (대출빙자형) type only.
2. Each conversation consists of 8 to 12 sentences, and the number of sentences should be evenly distributed across the dataset.
3. The scammer (0) and the victim (1) alternate in the conversation.
4. Only one conversation type per dialogue.
5. Always use the name 'OOO' in all conversations (e.g., "Hello, is this OOO?").
6. Make the conversations realistic, natural, and highly diverse. Vary the loan types and ensure the victim's responses are also diverse (suspicious, compliant, confused, resistant, etc.).
7. Avoid duplication with previously generated conversations.
8. Output the results in the following CSV format (without headers):

phishing_001,loan fraud,0,Hello, is this OOO? I am calling regarding a special loan offer.
phishing_001,loan fraud,1,Yes? What is this about?
(Generate 5 conversations from phishing_001 to phishing_005.)

Please output the conversation text in Korean.
""",
    "messenger phishing": """
You are a financial crime investigation expert with 15 years of experience and a developer of voice phishing detection systems.
Please create realistic and natural phone call scenarios for building a voice phishing detection dataset.

Rules:
1. Each conversation must be of the 'messenger phishing' (메신저피싱형) type only.
2. Each conversation consists of 8 to 12 sentences, and the number of sentences should be evenly distributed across the dataset.
3. The scammer (0) and the victim (1) alternate in the conversation.
4. Only one conversation type per dialogue.
5. Always use the name 'OOO' in all conversations (e.g., "Hello, is this OOO?").
6. Make the conversations realistic, natural, and highly diverse. Vary the impersonated messenger types and ensure the victim's responses are also diverse (suspicious, compliant, confused, resistant, etc.).
7. Avoid duplication with previously generated conversations.
8. Output the results in the following CSV format (without headers):

phishing_001,messenger phishing,0,Hello, is this OOO? I'm contacting you through your messenger app.
phishing_001,messenger phishing,1,Yes, this is OOO. Who is this?
(Generate 5 conversations from phishing_001 to phishing_005.)

Please output the conversation text in Korean.
""",
    "parcel scam": """
You are a financial crime investigation expert with 15 years of experience and a developer of voice phishing detection systems.
Please create realistic and natural phone call scenarios for building a voice phishing detection dataset.

Rules:
1. Each conversation must be of the 'parcel scam' (택배사기형) type only.
2. Each conversation consists of 13 to 15 sentences, and the number of sentences should be evenly distributed across the dataset.
3. The scammer (0) and the victim (1) alternate in the conversation.
4. Only one conversation type per dialogue.
5. Always use the name 'OOO' in all conversations (e.g., "Hello, is this OOO?").
6. Make the conversations realistic, natural, and highly diverse. Vary the delivery companies and ensure the victim's responses are also diverse (suspicious, compliant, confused, resistant, etc.).
7. Avoid duplication with previously generated conversations.
8. Output the results in the following CSV format (without headers):

phishing_001,parcel scam,0,Hello, is this OOO? I'm calling from the delivery service.
phishing_001,parcel scam,1,Yes, this is OOO. Is there a problem with my parcel?
(Generate 5 conversations from phishing_001 to phishing_005.)

Please output the conversation text in Korean.
"""
}

# 3. 기존 CSV에서 마지막 file_name의 번호를 읽어오는 함수
def get_next_start_index(save_path="./dataset/phishing_data.csv"):
    if os.path.exists(save_path):
        df = pd.read_csv(save_path)
        max_idx = (
            df["file_name"]
            .str.extract(r'phishing_(\d+)')[0]
            .dropna()
            .astype(int)
            .max()
        )
        return int(max_idx) + 1
    else:
        return 1

# 4. GPT-4o로 시나리오 생성
def generate_phishing_dialogues(n, start_index, scenario_type):
    user_prompt = (
        f"Please generate {n} {scenario_type} voice phishing scenarios. "
        f"File names should be assigned sequentially from phishing_{start_index:03d}."
    )
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt_dict[scenario_type]},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.8
    )
    return response.choices[0].message.content

# 5. 생성 결과를 CSV에 누적 저장(중복 제거)
def parse_and_save_csv_append_unique(response_text, save_path="./dataset/phishing_data.csv"):
    lines = [line.strip() for line in response_text.strip().split("\n") if line and not line.startswith("file_name")]
    data = [line.split(",", maxsplit=3) for line in lines if len(line.split(",", maxsplit=3)) == 4]
    df_new = pd.DataFrame(data, columns=["file_name", "conversation_type", "spk", "msg"])
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    if os.path.exists(save_path):
        df_existing = pd.read_csv(save_path)
        df_combined = pd.concat([df_existing, df_new], ignore_index=True)
        df_combined.drop_duplicates(inplace=True)
        df_combined.to_csv(save_path, index=False, encoding="utf-8-sig")
    else:
        df_new.to_csv(save_path, index=False, encoding="utf-8-sig")
    print(f"✅ CSV 저장 완료 및 중복 제거: {save_path}")

# 6. 실행
if __name__ == "__main__":
    print("🔄 보이스피싱 대화 생성 중...")
    try:
        scenario_plan = [
            ("authority impersonation", 40),
            ("loan fraud", 20),
            ("messenger phishing", 20),
            ("parcel scam", 20)
        ]
        batch_size = 5
        save_path = "./dataset/phishing_data.csv"
        start_index = get_next_start_index(save_path)
        for scenario_type, total_count in scenario_plan:
            for i in range(0, total_count, batch_size):
                print(f"생성 중: {scenario_type} {start_index} ~ {start_index+batch_size-1} (파일명 시작: phishing_{start_index:03d})")
                output = generate_phishing_dialogues(batch_size, start_index, scenario_type)
                parse_and_save_csv_append_unique(output, save_path=save_path)
                start_index += batch_size
    except RateLimitError:
        print("❌ RateLimitError: 쿼터 초과 또는 모델 접근 제한입니다. 잠시 후 재시도하거나 대시보드를 확인하세요.")
    except Exception as e:
        print(f"❌ 오류 발생: {e}")


🔄 보이스피싱 대화 생성 중...
생성 중: authority impersonation 701 ~ 705 (파일명 시작: phishing_701)
✅ CSV 저장 완료 및 중복 제거: ./dataset/phishing_data.csv
생성 중: authority impersonation 706 ~ 710 (파일명 시작: phishing_706)
✅ CSV 저장 완료 및 중복 제거: ./dataset/phishing_data.csv
생성 중: authority impersonation 711 ~ 715 (파일명 시작: phishing_711)
✅ CSV 저장 완료 및 중복 제거: ./dataset/phishing_data.csv
생성 중: authority impersonation 716 ~ 720 (파일명 시작: phishing_716)
✅ CSV 저장 완료 및 중복 제거: ./dataset/phishing_data.csv
생성 중: authority impersonation 721 ~ 725 (파일명 시작: phishing_721)
✅ CSV 저장 완료 및 중복 제거: ./dataset/phishing_data.csv
생성 중: authority impersonation 726 ~ 730 (파일명 시작: phishing_726)
✅ CSV 저장 완료 및 중복 제거: ./dataset/phishing_data.csv
생성 중: authority impersonation 731 ~ 735 (파일명 시작: phishing_731)
✅ CSV 저장 완료 및 중복 제거: ./dataset/phishing_data.csv
생성 중: authority impersonation 736 ~ 740 (파일명 시작: phishing_736)
✅ CSV 저장 완료 및 중복 제거: ./dataset/phishing_data.csv
생성 중: loan fraud 741 ~ 745 (파일명 시작: phishing_741)
✅ CSV 저장 완료 및 중복 제거: ./dataset/phis

In [ ]:
# GPT-4o 모델을 활용하여 보이스피싱 시나리오(기관사칭형, 대출빙자형, 메신저피싱형, 택배사기형)를 자동으로 생성하고, 
# 생성된 데이터를 중복 없이 CSV 파일로 저장하는 자동화 파이프라인입니다.
# 각 시나리오 유형별로 대화 문장 수(13~15문장)를 엄격하게 제한하며, 피싱범(0)과 피해자(1)가 번갈아 대화하고, 
# 한 대화에는 한 가지 유형만 포함됩니다.
# 대화 파일명은 phishing_001처럼 순차적으로 부여되고, 모든 대화에서 이름은 'OOO'만 사용합니다.
# 각 유형별로 사칭 기관·대출 종류·메신저·택배사 등과 피해자 반응(의심, 순응, 당황, 저항 등)이 다양하게 포함되도록 
# 프롬프트가 설계되어 있습니다.
# 생성된 데이터는 기존 CSV와 합쳐 중복을 제거한 뒤 저장되며, RateLimitError 등 예외 상황도 처리합니다.

import os
import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI, RateLimitError

# 1. 환경 변수 로드 및 API 키 확인
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("❌ API 키가 로드되지 않았습니다. .env 파일을 확인하세요.")

client = OpenAI(api_key=api_key)

# 2. 시나리오 유형별 시스템 프롬프트 (문장 수 조건 포함)
system_prompt_dict = {
    "authority impersonation": """
You are a financial crime investigation expert with 15 years of experience and a developer of voice phishing detection systems.
Please create realistic and natural phone call scenarios for building a voice phishing detection dataset.

Rules:
1. Each conversation must consist of 13 to 15 sentences. Conversations with fewer than 13 sentences are not allowed under any circumstances.
2. Each conversation must be of the 'authority impersonation' (기관사칭형) type only.
3. The scammer (0) and the victim (1) alternate in the conversation.
4. Only one conversation type per dialogue.
5. Always use the name 'OOO' in all conversations (e.g., "Hello, is this OOO?").
6. Make the conversations realistic, natural, and highly diverse. Vary the impersonated institutions (e.g., police, prosecutors, banks, tax office, insurance, etc.) and ensure the victim's responses are also diverse (suspicious, compliant, confused, resistant, etc.).
7. Avoid duplication with previously generated conversations.
8. Output the results in the following CSV format (without headers):

phishing_001,authority impersonation,0,Hello, is this OOO? This is Inspector Kim from the Seoul Metropolitan Police.
phishing_001,authority impersonation,1,Yes, that's me. What's going on?
(Generate 5 conversations from phishing_001 to phishing_005.)

Please output the conversation text in Korean.
""",
    "loan fraud": """
You are a financial crime investigation expert with 15 years of experience and a developer of voice phishing detection systems.
Please create realistic and natural phone call scenarios for building a voice phishing detection dataset.

Rules:
1. Each conversation must consist of 13 to 15 sentences. Conversations with fewer than 13 sentences are not allowed under any circumstances.
2. Each conversation must be of the 'loan fraud' (대출빙자형) type only.
3. The scammer (0) and the victim (1) alternate in the conversation.
4. Only one conversation type per dialogue.
5. Always use the name 'OOO' in all conversations (e.g., "Hello, is this OOO?").
6. Make the conversations realistic, natural, and highly diverse. Vary the loan types and ensure the victim's responses are also diverse (suspicious, compliant, confused, resistant, etc.).
7. Avoid duplication with previously generated conversations.
8. Output the results in the following CSV format (without headers):

phishing_001,loan fraud,0,Hello, is this OOO? I am calling regarding a special loan offer.
phishing_001,loan fraud,1,Yes? What is this about?
(Generate 5 conversations from phishing_001 to phishing_005.)

Please output the conversation text in Korean.
""",
    "messenger phishing": """
You are a financial crime investigation expert with 15 years of experience and a developer of voice phishing detection systems.
Please create realistic and natural phone call scenarios for building a voice phishing detection dataset.

Rules:
1. Each conversation must consist of 13 to 15 sentences. Conversations with fewer than 13 sentences are not allowed under any circumstances.
2. Each conversation must be of the 'messenger phishing' (메신저피싱형) type only.
3. The scammer (0) and the victim (1) alternate in the conversation.
4. Only one conversation type per dialogue.
5. Always use the name 'OOO' in all conversations (e.g., "Hello, is this OOO?").
6. Make the conversations realistic, natural, and highly diverse. Vary the impersonated messenger types and ensure the victim's responses are also diverse (suspicious, compliant, confused, resistant, etc.).
7. Avoid duplication with previously generated conversations.
8. Output the results in the following CSV format (without headers):

phishing_001,messenger phishing,0,Hello, is this OOO? I'm contacting you through your messenger app.
phishing_001,messenger phishing,1,Yes, this is OOO. Who is this?
(Generate 5 conversations from phishing_001 to phishing_005.)

Please output the conversation text in Korean.
""",
    "parcel scam": """
You are a financial crime investigation expert with 15 years of experience and a developer of voice phishing detection systems.
Please create realistic and natural phone call scenarios for building a voice phishing detection dataset.

Rules:
1. Each conversation must consist of 13 to 15 sentences. Conversations with fewer than 13 sentences are not allowed under any circumstances.
2. Each conversation must be of the 'parcel scam' (택배사기형) type only.
3. The scammer (0) and the victim (1) alternate in the conversation.
4. Only one conversation type per dialogue.
5. Always use the name 'OOO' in all conversations (e.g., "Hello, is this OOO?").
6. Make the conversations realistic, natural, and highly diverse. Vary the delivery companies and ensure the victim's responses are also diverse (suspicious, compliant, confused, resistant, etc.).
7. Avoid duplication with previously generated conversations.
8. Output the results in the following CSV format (without headers):

phishing_001,parcel scam,0,Hello, is this OOO? I'm calling from the delivery service.
phishing_001,parcel scam,1,Yes, this is OOO. Is there a problem with my parcel?
(Generate 5 conversations from phishing_001 to phishing_005.)

Please output the conversation text in Korean.
"""
}

# 3. 기존 CSV에서 마지막 file_name의 번호를 읽어오는 함수
def get_next_start_index(save_path="./dataset/phishing_data.csv"):
    if os.path.exists(save_path):
        df = pd.read_csv(save_path)
        max_idx = (
            df["file_name"]
            .str.extract(r'phishing_(\d+)')[0]
            .dropna()
            .astype(int)
            .max()
        )
        return int(max_idx) + 1
    else:
        return 1

# 4. GPT-4o로 시나리오 생성
def generate_phishing_dialogues(n, start_index, scenario_type):
    user_prompt = (
        f"Please generate {n} {scenario_type} voice phishing scenarios. "
        f"File names should be assigned sequentially from phishing_{start_index:03d}."
    )
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt_dict[scenario_type]},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.8
    )
    return response.choices[0].message.content

# 5. 생성 결과를 CSV에 누적 저장(중복 제거)
def parse_and_save_csv_append_unique(response_text, save_path="./dataset/phishing_data.csv"):
    lines = [line.strip() for line in response_text.strip().split("\n") if line and not line.startswith("file_name")]
    data = [line.split(",", maxsplit=3) for line in lines if len(line.split(",", maxsplit=3)) == 4]
    df_new = pd.DataFrame(data, columns=["file_name", "conversation_type", "spk", "msg"])
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    if os.path.exists(save_path):
        df_existing = pd.read_csv(save_path)
        df_combined = pd.concat([df_existing, df_new], ignore_index=True)
        df_combined.drop_duplicates(inplace=True)
        df_combined.to_csv(save_path, index=False, encoding="utf-8-sig")
    else:
        df_new.to_csv(save_path, index=False, encoding="utf-8-sig")
    print(f"✅ CSV 저장 완료 및 중복 제거: {save_path}")

# 6. 실행
if __name__ == "__main__":
    print("🔄 보이스피싱 대화 생성 중...")
    try:
        scenario_plan = [
            ("authority impersonation", 40),
            ("loan fraud", 20),
            ("messenger phishing", 20),
            ("parcel scam", 20)
        ]
        batch_size = 5
        save_path = "./dataset/phishing_data.csv"
        start_index = get_next_start_index(save_path)
        for scenario_type, total_count in scenario_plan:
            for i in range(0, total_count, batch_size):
                print(f"생성 중: {scenario_type} {start_index} ~ {start_index+batch_size-1} (파일명 시작: phishing_{start_index:03d})")
                output = generate_phishing_dialogues(batch_size, start_index, scenario_type)
                parse_and_save_csv_append_unique(output, save_path=save_path)
                start_index += batch_size
    except RateLimitError:
        print("❌ RateLimitError: 쿼터 초과 또는 모델 접근 제한입니다. 잠시 후 재시도하거나 대시보드를 확인하세요.")
    except Exception as e:
        print(f"❌ 오류 발생: {e}")


🔄 보이스피싱 대화 생성 중...
생성 중: authority impersonation 801 ~ 805 (파일명 시작: phishing_801)
✅ CSV 저장 완료 및 중복 제거: ./dataset/phishing_data.csv
생성 중: authority impersonation 806 ~ 810 (파일명 시작: phishing_806)
✅ CSV 저장 완료 및 중복 제거: ./dataset/phishing_data.csv
생성 중: authority impersonation 811 ~ 815 (파일명 시작: phishing_811)
✅ CSV 저장 완료 및 중복 제거: ./dataset/phishing_data.csv
생성 중: authority impersonation 816 ~ 820 (파일명 시작: phishing_816)
✅ CSV 저장 완료 및 중복 제거: ./dataset/phishing_data.csv
생성 중: authority impersonation 821 ~ 825 (파일명 시작: phishing_821)
✅ CSV 저장 완료 및 중복 제거: ./dataset/phishing_data.csv
생성 중: authority impersonation 826 ~ 830 (파일명 시작: phishing_826)
✅ CSV 저장 완료 및 중복 제거: ./dataset/phishing_data.csv
생성 중: authority impersonation 831 ~ 835 (파일명 시작: phishing_831)
✅ CSV 저장 완료 및 중복 제거: ./dataset/phishing_data.csv
생성 중: authority impersonation 836 ~ 840 (파일명 시작: phishing_836)
✅ CSV 저장 완료 및 중복 제거: ./dataset/phishing_data.csv
생성 중: loan fraud 841 ~ 845 (파일명 시작: phishing_841)
✅ CSV 저장 완료 및 중복 제거: ./dataset/phis